<div>
<img src="https://www.ul.ie/themes/custom/ul/logo.jpg" />
</div>

# **MSc in Artificial Intelligence and Machine Learning**
## CS6482 - Deep Reinforcement Learning
### Assignment 1: Sem2 AY 23/24 - Convolutational Neural Networks (CNNs)

Module Leader: J.J. Collins

Students:  
- Siddharth Prince - 23052058
- Pratik Verma - 23007575

---
## Code checks
### Test cases
 - Imports and setup - <span style='color:orange'>waiting</span>
 - Data preprocessing and EDA - <span style='color:orange'>waiting</span></li>
 - CNN architecture definition - <span style='color:orange'>waiting</span>
 - Training - <span style='color:orange'>waiting</span>
 - Testing - <span style='color:orange'>waiting</span>
 - Metrics and evalutation - <span style='color:orange'>waiting</span>
 - Finetuning model - <span style='color:orange'>waiting</span>
 - Engineering over-fitting - <span style='color:orange'>waiting</span>
 - Engineering under-fitting - <span style='color:orange'>waiting</span>
---

## 1. Imports and setup

In [13]:
import torch
from torch import nn # Importing torch's neural network (nn) class
import torch.nn.functional as F # importing this separately as 'F' to save the hasle of typing all that out. Also, it's funny.
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import Caltech101
from torchvision.transforms import ToTensor, Normalize, transforms
from torchsummary import summary
import matplotlib.pyplot as plt # Old faithful for plotting
from torchvision import models
import numpy as np # It isn't a jupyter notebook running ML models without numpy, is it?
import warnings 
warnings.filterwarnings('ignore') # To ignore the big red future warnings (we'll change our ways when things break and not premtively. 'Tis human nature.)

torch.manual_seed(69) # Setting a manual seed for consistency

import gdown # This required to download from the torch dataset library to account for gdrive API changes

### Selecting the device to use

In [6]:
# Get appropriate device for training
device = ( "cuda"     # CUDA to take advantage of Nvidia GPUs
           if torch.cuda.is_available()
           else "cpu" )   # If this fails -> CPU. Will probably use Google Colab and get a GPU.

print(f"{device} device available")

cuda device available


## 2. Data preprocessing and EDA

In [14]:
# Downloading the Caltech-101 dataset from the torch datasets library

caltech101_data = Caltech101(root='./content/', download=True)
data_loader = DataLoader(caltech101_data, # Loading the data into pytorch's Dataloader which is an iterable object making it easy to access the data
                         batch_size=16, # Defining the size for each batch
                         shuffle=True) # Shuffling the images before getting batched to remove any possible biases occoring from the order of the dataset

2439it [00:00, 28259412.86it/s]


RuntimeError: The MD5 checksum of the download file ./content/caltech101/101_ObjectCategories.tar.gz does not match the one on record.Please delete the file and try again. If the issue persists, please report this to torchvision at https://github.com/pytorch/vision/issues.